In [240]:
import pandas as pd
import numpy as np

## Reading data

In [241]:
large = False

if large:
    path = "market_value model\large"
else:
    path = "market_value model"

shooting = pd.read_csv(path + r"\mv-simulation-shooting.csv", index_col=0)
passing = pd.read_csv(path + r"\mv-simulation-passing.csv", index_col=0)
defense = pd.read_csv(path + r"\mv-simulation-defense.csv", index_col=0)
goalkeeping = pd.read_csv(path + r"\mv-simulation-goalkeeping.csv", index_col=0)

player_shooting = pd.read_csv("..\data\processed-data\l_shooting_clean.csv", index_col=0)
player_passing = pd.read_csv("..\data\processed-data\l_passing_clean_90.csv", index_col=0)
player_defense = pd.read_csv("..\data\processed-data\l_defense_clean.csv", index_col=0)
player_goalkeeping = pd.read_csv("..\data\processed-data\l_goalkeeping_clean.csv", index_col=0)

all_players = pd.concat([
    player_shooting[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_passing[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_defense[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    player_goalkeeping[['Player', 'Nation', 'Squad', 'Age', 'Year', 'FW', 'MF', 'DF', 'GK']],
    ])
all_players = all_players[all_players.Year == 2021].drop_duplicates().reset_index(drop=True)
print(all_players.shape)
all_players = all_players.assign(Raritan = lambda x: x.Nation == "Rarita")

(2822, 9)


In [242]:
print(all_players[all_players[["Player", "Nation", "Age"]].duplicated()].sort_values(by = "Player").shape)

all_players_group = all_players.groupby(["Player", "Nation", "Age"])
all_players = all_players_group[['FW', 'MF', 'DF', 'GK', 'Raritan']].any().reset_index()
print(all_players.shape)

(105, 10)
(2717, 8)


In [243]:
trials = max(shooting.trial) + 1

# Remove duplicates in players
shooting = shooting.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
passing = passing.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
defense = defense.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()
goalkeeping = goalkeeping.groupby(["Player", "Nation", "trial"])[["mv0", "mv1", "mv2", "mv3", "mv4", "mv5", "mv6", "mv7", "mv8", "mv9"]].median().reset_index()

In [294]:
pd.options.display.max_rows = 1000
all_players[["FW","MF", "DF", "GK", "Raritan"]] = all_players[["FW","MF", "DF", "GK", "Raritan"]].astype(int)
all_players.to_csv(r"..\data\optim-data\0_selection_players.csv")
all_players

PermissionError: [Errno 13] Permission denied: '..\\data\\optim-data\\0_selection_players.csv'

In [275]:
all_players['player_id'] = all_players.apply(lambda row: "-".join([row[0], row[1]]), axis=1)

shooting['player_id'] = shooting.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
passing['player_id'] = passing.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
defense['player_id'] = defense.apply(lambda row: "-".join([row[0], row[1]]), axis=1)
goalkeeping['player_id'] = goalkeeping.apply(lambda row: "-".join([row[0], row[1]]), axis=1)

shooting.index = pd.MultiIndex.from_frame(shooting[['player_id', 'trial']])
passing.index = pd.MultiIndex.from_frame(passing[['player_id', 'trial']])
defense.index = pd.MultiIndex.from_frame(defense[['player_id', 'trial']])
goalkeeping.index = pd.MultiIndex.from_frame(goalkeeping[['player_id', 'trial']])

In [290]:
from itertools import product
player_mv = pd.DataFrame()
player_mv.index = pd.MultiIndex.from_tuples(list(product(all_players.player_id, np.arange(trials))))

for i in all_players.index:
    if (i % 100 == 0):
        print(f"{i} out of {all_players.shape[0]}")

    player_id = "-".join([all_players.iloc[i, 0], all_players.iloc[i, 1]])
    
    role_FW = all_players.iloc[i, 3]
    role_MF = all_players.iloc[i, 4]
    role_DF = all_players.iloc[i, 5]
    role_GK = all_players.iloc[i, 6]

    best_role = -1
    mv = float("-Inf")
    best_df = None
    for role_i, role in enumerate([role_FW, role_MF, role_DF, role_GK]):
        if role_i == 0:
            df = shooting
        elif role_i == 1:
            df = passing
        elif role_i == 2:
            df = defense
        elif role_i == 3:
            df = goalkeeping
        
        try:
            if best_role == -1 or (best_role != -1 and df.loc[(player_id, 0), "mv0"] > mv):
                mv = df.loc[(player_id, 0), "mv0"]
                best_role = role_i
                best_df = df
        except KeyError:
            pass
    
    for j in np.arange(trials):
        for t in np.arange(10):
            mv_t = "mv" + str(t)
            
            try:
                player_mv.loc[(player_id, j), mv_t] = best_df.loc[(player_id, j), mv_t]
            except:
                break

player_mv = player_mv.reset_index()

0 out of 2717
100 out of 2717
200 out of 2717
300 out of 2717
400 out of 2717
500 out of 2717
600 out of 2717
700 out of 2717
800 out of 2717
900 out of 2717
1000 out of 2717
1100 out of 2717
1200 out of 2717
1300 out of 2717
1400 out of 2717
1500 out of 2717
1600 out of 2717
1700 out of 2717
1800 out of 2717
1900 out of 2717
2000 out of 2717
2100 out of 2717
2200 out of 2717
2300 out of 2717
2400 out of 2717
2500 out of 2717
2600 out of 2717
2700 out of 2717


In [316]:
player_mv = player_mv.fillna(0)
player_mv = player_mv.rename(columns={"level_0": "player_id", "level_1": "trial"})
player_mv = player_mv.sort_values(by = ["trial", "player_id"])
player_mv.to_csv(r"..\data\optim-data\0_selection_mv.csv")

In [317]:
player_mv

,player_id,trial,mv0,mv1,mv2,mv3,mv4,mv5,mv6,mv7,mv8,mv9
0,a.aceng-Esia,0,13800000.0,12800000.0,11700000.0,10500000.0,9100000.0,7800000.0,6600000.0,5400000.0,4300000.0,3400000.0
50,a.acheng-Southern Ristan,0,39400000.0,36600000.0,33600000.0,30500000.0,27200000.0,24000000.0,21000000.0,18000000.0,15300000.0,12800000.0
100,a.agyapong-Imaar Vircoand,0,9500000.0,9200000.0,8700000.0,8100000.0,7400000.0,6600000.0,5800000.0,5000000.0,4200000.0,3400000.0
150,a.ainomugisha-Moaithe,0,36000000.0,36500000.0,36200000.0,35100000.0,33400000.0,31100000.0,28400000.0,25300000.0,22100000.0,19000000.0
200,a.akoth-Unicorporated Tiagascar,0,13050000.0,11400000.0,9800000.0,8200000.0,6700000.0,5400000.0,4250000.0,3250000.0,2500000.0,1800000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
135649,z.vilanculos-Esia,49,9700000.0,73900000.0,38600000.0,20200000.0,4300000.0,3900000.0,1600000.0,23500000.0,3100000.0,3300000.0
135699,z.wati-Dosqaly,49,13800000.0,2500000.0,1500000.0,4000000.0,2000000.0,1700000.0,4100000.0,2400000.0,4000000.0,1700000.0
135749,z.wu-Ledian,49,11700000.0,162500000.0,54800000.0,27100000.0,15500000.0,57300000.0,40200000.0,19900000.0,12400000.0,26500000.0
135799,z.yosef-People's Land of Maneau,49,9400000.0,250000.0,12200000.0,45000000.0,15400000.0,7300000.0,13000000.0,3100000.0,3400000.0,4500000.0


50.0